In [34]:
import requests
from pymongo import MongoClient
import pyspark as ps
from bs4 import BeautifulSoup

In [4]:
# Uses all 4 cores on your machine
sc = ps.SparkContext('local[4]')

In [28]:
class DataCollector(object):

    def __init__(self, collection_name):
        
        self.TYPE_USER = 'user'
        self.TYPE_ORG = 'org'
        self.collection_name = collection_name
        # Get MongoDB
        mc = MongoClient()
        self.db = mc['changeorg']
        self.petitions_col = self.db.collection_name


    def get_htmls(self, petition_url, creator_url, petition_id, creator_type):
        collection = self.db[self.collection_name + "_html"]
        #get petition html
        petition_response = requests.get(creator_url)
        petition_html = petition_response.content
        #get creator html
        creator_response = requests.get(petition_url)
        creator_html = creator_response.content
        #save backup of html in mongodb
        htmls_dict = {'petition_id': petition_id, 'petition_url': petition_url, 
                      'creator_url': creator_url, 'petition_html':html, 
                      'creator_html': creator_html, 'creator_type': creator_type}
        collection.insert(htmls_dict)
        
        return (petition_html, creator_html)

    def get_detailed_data(self):
        petitions = self.petitions_col.find().limit(2)
        for petition in petitions:
            print petition["_id"]
            petition_id = petition["petition_id"]
            #Get html
            self.get_html(petition["url"], petition_id, self.TYPE_PETITION)
            # Get user
            creator_url = petition["creator_url"]
            org_url = petition["organization_url"]
            if creator_url: 
                creator_type = self.TYPE_USER               
            elif org_url: 
                creator_type =  self.TYPE_ORG
            html = self.get_html(org_url, petition_id, creator_type)
            new_fields = self.webscrape(petition, petition_html, creator_html, creator_type)
            self.petitions_col.update(
                {'_id': petition['_id']},
                    {
                     '$set': new_fields
                }, upsert=False)
    def webscrape(self, petition, petition_html, creator_html, creator_type):
        new_fields = {}
        
        return new_fields



In [29]:
dc = DataCollector("sample_us_closed_petitions")

In [30]:
dc.get_detailed_data()

{u'status': u'closed', u'category': u'Environment', u'organization_url': None, u'goal': 5000, u'creator_name': u'Christopher David Moore', u'title': u"Freeze all of BP's assets, before it is too late", u'url': u'https://api.change.org/p/freeze-all-of-bp-s-assets-before-it-is-too-late', u'overview': u'<p>In the construction industry, builders that get sued for one reason or another simply drop their company name to escape paying law suits. It only takes 6 hours to\xa0make a new company.(REBRANDING) Same owner, same company practices, different name and zero responsibility to the lawsuits of prior company.</p>\n<p>This could be the reason BP is slow to act. Stall tactics so they can move things around. False accounts to how much oil is spewing out daily, because the amount matters in terms of fines levied against them.</p>\n<p>We must act NOW. WE MUST ACT NOW BEFORE BP RUNS AND HIDES UNDER ANOTHER NAME.</p>\n<p>FREEZE ALL THAT IS CONNECTED TO BP!\xa0\xa0THE OWNERS ASSETS TOO, \xa0just as